# Transfer learning with PyTorch
We're going to train a neural network to classify dogs and cats.

## Init, helpers, utils, ...

In [1]:
from pprint import pprint
from IPython.core.debugger import set_trace

import matplotlib.pyplot as plt
import numpy as np

from ppt import utils
from ppt.utils import attr

%matplotlib inline

In [2]:
# Training helpers
def get_trainable(model_params):
    return (p for p in model_params if p.requires_grad)


def get_frozen(model_params):
    return (p for p in model_params if not p.requires_grad)


def all_trainable(model_params):
    return all(p.requires_grad for p in model_params)


def all_frozen(model_params):
    return all(not p.requires_grad for p in model_params)


def freeze_all(model_params):
    for param in model_params:
        param.requires_grad = False


# list(get_trainable(model.parameters()))
# list(get_frozen(model.parameters()))
# all_trainable(model.parameters())
# all_frozen(model.parameters())

# The Data

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Transforms

In [5]:
from torchvision import transforms

IMG_SIZE = 224
_mean = [0.485, 0.456, 0.406]
_std = [0.229, 0.224, 0.225]


train_trans = transforms.Compose([
    transforms.Resize(256),  # some images are pretty small
    transforms.RandomCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(.3, .3, .3),
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])
val_trans = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])

## Dataset

In [6]:
from ppt.utils import DogsCatsDataset

train_ds = DogsCatsDataset("../data/raw", "train", transform=train_trans)
val_ds = DogsCatsDataset("../data/raw", "valid", transform=val_trans)
# train_ds = DogsCatsDataset("../data/raw", "sample/train", transform=train_trans)
# val_ds = DogsCatsDataset("../data/raw", "sample/valid", transform=val_trans)


n_classes = 2

Loading data from ../data/raw/dogscats/train.
Loading data from ../data/raw/dogscats/valid.


In [7]:
len(train_ds), len(val_ds)

(23000, 2000)

## DataLoader
Batch loading for datasets with multi-processing and different sample strategies.

In [8]:
from torch.utils.data import DataLoader

BATCH_SIZE = 128
# BATCH_SIZE = 512

train_dl = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
)
val_dl = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
)

# The Model
PyTorch offers quite a few [pre-trained networks](https://pytorch.org/docs/stable/torchvision/models.html) for you to use:
- AlexNet
- VGG
- ResNet
- SqueezeNet
- DenseNet
- Inception v3

And there are more available via [pretrained-models.pytorch](https://github.com/Cadene/pretrained-models.pytorch)
- NASNet,
- ResNeXt,
- InceptionV4,
- InceptionResnetV2, 
- Xception, 
- DPN,
- ...

In [9]:
from torchvision import models

model = models.resnet18(pretrained=True)

In [10]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [11]:
# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

In [12]:
freeze_all(model.parameters())
assert all_frozen(model.parameters())

Replace the last layer with a linear layer. New layers have `requires_grad = True`.

In [13]:
model.fc = nn.Linear(512, n_classes)

In [14]:
all_frozen(model.parameters())

False

In [15]:
def get_model(n_classes=2):
    model = models.resnet18(pretrained=True)
    freeze_all(model.parameters())
    model.fc = nn.Linear(512, n_classes)
    return model


model = get_model().to(device)

# The Loss

In [16]:
criterion = nn.CrossEntropyLoss()

# The Optimizer

In [17]:
optimizer = torch.optim.Adam(
    get_trainable(model.parameters()),  # model.fc.parameters()
    lr=0.001,
    # momentum=0.9,
)

# The Train Loop

In [18]:
N_EPOCHS = 2

In [19]:
for epoch in range(N_EPOCHS):
    print(f"Epoch {epoch+1}/{N_EPOCHS}")
    
    # Train
    model.train()  # IMPORTANT
    
    running_loss, correct = 0.0, 0
    for X, y in train_dl:
        X, y = X.to(device), y.to(device)
        
        optimizer.zero_grad()
        # with torch.set_grad_enabled(True):
        y_ = model(X)
        loss = criterion(y_, y)

        loss.backward()
        optimizer.step()
        
        # Statistics
        print(f"    batch loss: {loss.item():0.3f}")
        _, y_label_ = torch.max(y_, 1)
        correct += (y_label_ == y).sum().item()
        running_loss += loss.item() * X.shape[0]
    
    print(f"  Train Loss: {running_loss / len(train_dl.dataset)}")
    print(f"  Train Acc:  {correct / len(train_dl.dataset)}")
    
    
    # Eval
    model.eval()  # IMPORTANT
    
    running_loss, correct = 0.0, 0
    with torch.no_grad():  # IMPORTANT
        for X, y in val_dl:
            X, y = X.to(device), y.to(device)
                    
            y_ = model(X)
        
            _, y_label_ = torch.max(y_, 1)
            correct += (y_label_ == y).sum().item()
            
            loss = criterion(y_, y)
            running_loss += loss.item() * X.shape[0]
    
    print(f"  Valid Loss: {running_loss / len(val_dl.dataset)}")
    print(f"  Valid Acc:  {correct / len(val_dl.dataset)}")
    print()

Epoch 1/2
    batch loss: 0.842
    batch loss: 0.742
    batch loss: 0.677
    batch loss: 0.644
    batch loss: 0.637
    batch loss: 0.536
    batch loss: 0.494
    batch loss: 0.476
    batch loss: 0.479
    batch loss: 0.426
    batch loss: 0.425
    batch loss: 0.363
    batch loss: 0.380
    batch loss: 0.341
    batch loss: 0.350
    batch loss: 0.310
    batch loss: 0.384
    batch loss: 0.260
    batch loss: 0.308
    batch loss: 0.252
    batch loss: 0.281
    batch loss: 0.273
    batch loss: 0.236
    batch loss: 0.222
    batch loss: 0.169
    batch loss: 0.238
    batch loss: 0.216
    batch loss: 0.229
    batch loss: 0.235
    batch loss: 0.189
    batch loss: 0.214
    batch loss: 0.202
    batch loss: 0.263
    batch loss: 0.154
    batch loss: 0.193
    batch loss: 0.200
    batch loss: 0.184
    batch loss: 0.197
    batch loss: 0.158
    batch loss: 0.178
    batch loss: 0.187
    batch loss: 0.162
    batch loss: 0.171
    batch loss: 0.171
    batch loss: 0.174


## Intermission: training libraries

Writing the training loop is my least favourite thing about PyTorch.

Keras is great here!
```python
model.compile(optimizer, criterion, metrics=["accuracy", "f1"])
model.fit(X, y, epochs=10)
```


### [Ignite](https://github.com/pytorch/ignite)
> Ignite is a high-level library to help with training neural networks in PyTorch.
> - ignite helps you write compact but full-featured training loops in a few lines of code
> - you get a training loop with metrics, early-stopping, model checkpointing and other features without the boilerplate


### [TNT](https://github.com/pytorch/tnt)
> TNT is a library providing powerful dataloading, logging and visualization utlities for Python. It is closely intergrated with PyTorch and is designed to enable rapid iteration with any model or training regimen.
> [...]
> The project was inspired by TorchNet, and legend says that it stood for “TorchNetTwo”


### [Skorch](https://github.com/dnouri/skorch)
> A scikit-learn compatible neural network library that wraps PyTorch.


### "The fun of Reinvention"
Clearly, there must be a better way! Write your own lib (but don't use it) :D

## Demo with Ignite

In [20]:
import ignite
from ignite.metrics import (
    CategoricalAccuracy,
    Loss,
    Precision,
)
from ignite.engine import (
    create_supervised_evaluator,
    create_supervised_trainer,
    Events,
)

In [21]:
# model, loss, optimizer
model = get_model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    get_trainable(model.parameters()),
    lr=0.001,
    momentum=.9,
)

# trainer and evaluator
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(
    model,
    metrics={
        "accuracy": CategoricalAccuracy(),
        "loss": Loss(criterion),
        "precision": Precision(),
    },
    device=device,
)

In [22]:
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    print(f"Epoch[{engine.state.epoch}] Batch[{engine.state.iteration}] Loss: {engine.state.output:.2f}")


# trainer.run(train_dl, max_epochs=1)

In [23]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_dl)
    metrics = evaluator.state.metrics
    print(f"Training Results   - Epoch: {trainer.state.epoch}  "
          f"accuracy: {metrics['accuracy']:.2f} "
          f"loss: {metrics['loss']:.2f} "
          f"prec: {metrics['precision'].cpu()}")


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(val_dl)
    metrics = evaluator.state.metrics
    print(f"Validation Results - Epoch: {trainer.state.epoch}  "
          f"accuracy: {metrics['accuracy']:.2f} "
          f"loss: {metrics['loss']:.2f} "
          f"prec: {metrics['precision'].cpu()}")


trainer.run(train_dl, max_epochs=1)

Epoch[1] Batch[1] Loss: 0.75
Epoch[1] Batch[2] Loss: 0.68
Epoch[1] Batch[3] Loss: 0.68
Epoch[1] Batch[4] Loss: 0.61
Epoch[1] Batch[5] Loss: 0.69
Epoch[1] Batch[6] Loss: 0.62
Epoch[1] Batch[7] Loss: 0.62
Epoch[1] Batch[8] Loss: 0.57
Epoch[1] Batch[9] Loss: 0.55
Epoch[1] Batch[10] Loss: 0.50
Epoch[1] Batch[11] Loss: 0.45
Epoch[1] Batch[12] Loss: 0.47
Epoch[1] Batch[13] Loss: 0.40
Epoch[1] Batch[14] Loss: 0.42
Epoch[1] Batch[15] Loss: 0.35
Epoch[1] Batch[16] Loss: 0.33
Epoch[1] Batch[17] Loss: 0.30
Epoch[1] Batch[18] Loss: 0.29
Epoch[1] Batch[19] Loss: 0.27
Epoch[1] Batch[20] Loss: 0.29
Epoch[1] Batch[21] Loss: 0.28
Epoch[1] Batch[22] Loss: 0.27
Epoch[1] Batch[23] Loss: 0.27
Epoch[1] Batch[24] Loss: 0.27
Epoch[1] Batch[25] Loss: 0.25
Epoch[1] Batch[26] Loss: 0.22
Epoch[1] Batch[27] Loss: 0.22
Epoch[1] Batch[28] Loss: 0.22
Epoch[1] Batch[29] Loss: 0.24
Epoch[1] Batch[30] Loss: 0.19
Epoch[1] Batch[31] Loss: 0.17
Epoch[1] Batch[32] Loss: 0.23
Epoch[1] Batch[33] Loss: 0.17
Epoch[1] Batch[34] 

# Visualization with Tensorboard and TensorboardX
- https://www.tensorflow.org/programmers_guide/summaries_and_tensorboard
- https://github.com/lanpa/tensorboard-pytorch

Start tensorboard:
```
cd notebooks
tensorboard --logdir=tf_log
```

In [24]:
from tensorboardX import SummaryWriter

summary_writer = SummaryWriter(log_dir="tf_log/exp1")

In [25]:
ls tf_log

exp1/  exp1_ignite/


In [26]:
# Visualize the graph/network
X, _ = next(iter(train_dl))
summary_writer.add_graph(model, X)

Error occurs, checking if it's onnx problem...
Your model fails onnx too, please report to onnx team
No graph saved


Then visit http://localhost:6006

In [27]:
# write some scalars
for i in range(10):
    summary_writer.add_scalar("training/loss", np.random.rand(), i)
    summary_writer.add_scalar("validation/loss", np.random.rand() + .1, i)

Use tensorboard with ignite

In [28]:
# new SummaryWriter for new experiment
summary_writer = SummaryWriter(log_dir="tf_log/exp1_ignite")

# Basic setup: model, loss, optimizer
model = get_model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(get_trainable(model.parameters()), lr=0.001, momentum=.9)

# trainer and evaluator
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(
    model,
    metrics={
        "accuracy": CategoricalAccuracy(),
        "loss": Loss(criterion),
    },
    device=device,
)

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_dl)
    metrics = evaluator.state.metrics
    epoch = trainer.state.epoch
    summary_writer.add_scalar("training/accuracy", metrics['accuracy'], epoch)
    summary_writer.add_scalar("training/loss", metrics['loss'], epoch)


@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(val_dl)
    metrics = evaluator.state.metrics
    epoch = trainer.state.epoch
    summary_writer.add_scalar("validation/accuracy", metrics['accuracy'], epoch)
    summary_writer.add_scalar("validation/loss", metrics['loss'], epoch)
    print(metrics['accuracy'])

In [29]:
trainer.run(train_dl, 4)

0.9735
0.975
0.9765
0.976


## Intermission: the output of a model

# Write your own dataset

In [30]:
from torch.utils.data import Dataset


class MyData(Dataset):
    def __init__(self):
        super().__init__()
        
    def __len__(self):
        return 0
    
    def __getitem__(self, idx):
        return None

# PyTorch Outlook
- [0.4 "just" released](https://pytorch.org/2018/04/22/0_4_0-migration-guide.html)
  - `Tensors` and `Variables` have merged
  - Support for 0-dimensional (scalar) Tensors
  - Deprecation of the `volatile` flag
  
- [The road to 1.0: production ready PyTorch](https://pytorch.org/2018/05/02/road-to-1.0.html)
  - `torch.jit`
  - optimize for mobile
  - quantized inference (such as 8-bit inference)
  - caffe2 merget into pytorch repo
  - ONNX - Open Neural Network Exchange

In [31]:
import torch.jit

In [32]:
from torch.jit import script

@script
def rnn_loop(x):
    hidden = None
    for x_t in x.split(1):
        x, hidden = model(x, hidden)
    return x

RuntimeError: 
unknown builtin op:
@script
def rnn_loop(x):
    hidden = None
    for x_t in x.split(1):
               ~~~~~~~ <--- HERE
        x, hidden = model(x, hidden)
    return x
